## Baaseline Span Identification

In [21]:
import pandas as pd
import os
import random
import ast
import csv

In [3]:
path = 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets'
train = pd.read_csv(os.path.join(path, 'input_train.csv'))
dev = pd.read_csv(os.path.join(path, 'input_dev.csv'))

train

,message_id,message_clean,annotation_new
0,209751,"доброго вечора, ми з україни – легендарний ви...","[{'name': 'Emotion_other', 'start': 32, 'end':..."
1,209779,мільйон гривень за шкільний туалет витратять у...,"[{'name': 'Doubts_government', 'start': 358, '..."
2,209784,вручення повісток у спортзалах є законним: кач...,"[{'name': 'Black_White', 'start': 76, 'end': 2..."
3,209785,окупанти продовжують терор мирного населення х...,"[{'name': 'Emotion_Fear', 'start': 21, 'end': ..."
4,209789,"загинув чоловік через обстріл вовчанська, – ов...","[{'name': 'Emotion_Fear', 'start': 215, 'end':..."
...,...,...,...
1328,1636641,"сьогодні у київраді проходить чергова сесія, д...","[{'name': 'Emotion_Anger', 'start': 129, 'end'..."
1329,1636679,що розповів зеленський на пресконференції? зіб...,"[{'name': 'Doubts_government', 'start': 800, '..."
1330,1636698,"""орбан"" із гааги: чи зможе противник допомоги ...","[{'name': 'Emotion_other', 'start': 413, 'end'..."
1331,1637606,скандал із співробітником віськомату в запоріж...,"[{'name': 'Doubts_Army', 'start': 126, 'end': ..."


In [4]:
print(f"Posts:", train.shape)
print(f"Posts:", dev.shape)

Posts: (1333, 3)
Posts: (572, 3)


#### Generate random span and check overlaping on character level

In [20]:
# Overlaping on character level
overlaps = []

for index, row in dev.iterrows():
    text = row["message_clean"]
    true_spans = ast.literal_eval(row["annotation_new"])
    span_found = False
        
    # Randomly generate start and end positions within the length of the text
    # Ensure end is greater than start

    start = random.randint(0, len(text)-1)
    end = random.randint(start + 1, len(text))
    
    # Calculate the length of the randomly generated span
    span_length = end - start
    
    max_overlap = 0
    
    # Compare the generation and truth
    for true_span in true_spans:   
        true_start = true_span["start"]
        true_end = true_span["end"]
    
        overlap = max(0, min(end, true_end) - max(start, true_start))
        max_overlap = max(max_overlap, overlap)
        
    overlaps.append(max_overlap)

# Calculate the overall accuracy
average_overlap = sum(overlaps) / len(overlaps)
print(f"Accuracy on character level: {average_overlap:.2f}")

Accuracy on character level: 10.42


#### Human-level perfomance

In [29]:
human = 'C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/SI.labels_human.txt'
true_file = "C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/my_master/datasets/SI.labels_true.txt"

In [ ]:
def merge_overlapping(indices_list):
  """
  Merges overlapping indices and sorts indices from list of tuples
  """
  # If no propaganda, return empty list
  if indices_list == []:
    return []

  # Sort the list
  indices_list = sorted(indices_list)
  i = 0

  # Going through tuples from the beginning, see if it overlaps with the nex one 
  # and merge it if so
  while True:
    if i == len(indices_list)- 1: ###!!!! - 1
      break
    
    # If the next one is within the range of current, just delete next one. If
    # overlapping, then merge the range
    elif indices_list[i][1] >= indices_list[i + 1][0]:
      if indices_list[i][1] >= indices_list[i + 1][1]:
        pass
      else:
        indices_list[i] = (indices_list[i][0], indices_list[i+1][1])
      indices_list.pop(i+1)

    # Go to next element if not overlapping
    else:
      i += 1

  return indices_list

In [ ]:
def articles_to_dataframe(article_folder, label_folder):
  """
  Preprocesses the articles into dataframes with sequences with binary tags
  """
  # First sort the filenames and make sure we have label file for each articles
  article_filenames = sorted(glob.glob(os.path.join(article_folder, "*.txt")))
  label_filenames = sorted(glob.glob(os.path.join(label_folder, "*.labels.tsv"))) #!!! format
  assert len(article_filenames) == len(label_filenames)

  # Initialize sequences
  sequences = []

  # For each article, do:
  for i in range(len(article_filenames)):
    # Get the id name
    article_id = os.path.basename(article_filenames[i]).split(".")[0][7:]

    # Read in the article
    with codecs.open(article_filenames[i], "r", encoding="utf8") as f:
      article = f.read()

    # Read in the label file and store indices for SI task

    with open(label_filenames[i], "r") as f:
      reader = csv.reader(f, delimiter="\t")
      indices_list = []
      for row in reader:
        indices_list.append([int(row[1]), int(row[2])])

      # Merge the indices if overlapping
      indices_list = merge_overlapping(indices_list)

    # Add to the sequences
    sequences.append(article_labels_to_sequences(article, indices_list))

  # Concatenate all dataframes
  dataframe = pd.concat(sequences, ignore_index=True)

  return dataframe

In [30]:
# Initialize dictionaries to store IDs and techniques
human_dict = {}
true_dict = {}

# Read contents of TC.labels_pred.txt
with open(human, "r") as pred_f:
    pred_reader = csv.reader(pred_f, delimiter="\t")
    for row in pred_reader:
      if len(row)>=3:
        id_ = row[0]
        start = int(row[1])
        end = int(row[2])
        if id_ not in human_dict:
            human_dict[id_] = [(start, end)]
        else:
            human_dict[id_].append((start, end))
      # else:
      # # Handle cases where the row doesn't have enough elements
      #   print("Row does not have enough elements:", row)
print (len(human_dict)) 

#Read contents of TC.labels_true.txt
with open(true_file, "r") as true_f:
    true_reader = csv.reader(true_f, delimiter="\t")
    for row in true_reader:
        id_ = row[0]
        start = int(row[1])
        end = int(row[2])
        if id_ not in true_dict:
            true_dict[id_] = [(start, end)]
        else:
            true_dict[id_].append((start, end))
print (len(true_dict)) 

349
572


In [26]:
human_dict

{'209751': [(133, 225)],
 '209784': [(267, 281), (76, 228), (43, 73)],
 '209879': [(136, 186)],
 '211938': [(273, 274),
  (276, 278),
  (280, 284),
  (286, 287),
  (289, 295),
  (121, 127)],
 '212219': [(0, 10),
  (12, 18),
  (20, 26),
  (28, 31),
  (179, 184),
  (203, 209),
  (90, 96),
  (164, 168)],
 '212301': [(9, 11), (13, 16), (59, 64), (89, 98)],
 '315436': [(140, 147),
  (149, 156),
  (160, 164),
  (166, 173),
  (175, 178),
  (180, 183),
  (191, 197),
  (185, 189),
  (203, 212),
  (214, 218),
  (249, 253),
  (255, 261),
  (263, 270)],
 '315445': [(185, 187),
  (189, 192),
  (196, 200),
  (202, 205),
  (208, 218),
  (125, 126),
  (224, 226)],
 '315580': [(271, 273),
  (275, 277),
  (281, 283),
  (285, 289),
  (291, 301),
  (0, 5),
  (7, 15),
  (17, 23),
  (25, 32),
  (34, 38),
  (40, 45),
  (47, 51)],
 '315640': [(0, 4),
  (6, 10),
  (83, 89),
  (91, 94),
  (99, 104),
  (106, 116),
  (120, 127),
  (132, 135),
  (33, 34),
  (191, 197),
  (33, 34),
  (202, 207),
  (185, 189)],
 '31

In [34]:
human_ids = pd.read_csv('C:/N/st/UCU/5sem2024_DIPLOMA/Manipulation/code/labels/human_evaluation_ids.csv')
human_ids = human_ids["message_id"].tolist()
human_ids

[209751,
 209784,
 209789,
 209879,
 209963,
 211938,
 212219,
 212301,
 315436,
 315445,
 315580,
 315640,
 315776,
 315785,
 315800,
 315803,
 315838,
 316248,
 316669,
 316750,
 316833,
 321562,
 321570,
 321579,
 321591,
 321596,
 321611,
 321641,
 321703,
 321731,
 321742,
 321754,
 321798,
 321848,
 321855,
 321891,
 321976,
 322004,
 322102,
 322179,
 322196,
 322280,
 322290,
 322300,
 322316,
 322455,
 322566,
 322723,
 322745,
 322796,
 322817,
 322863,
 322869,
 323103,
 323176,
 323299,
 323322,
 323371,
 327213,
 327768,
 334143,
 334548,
 334747,
 342100,
 358912,
 359876,
 360287,
 373628,
 379915,
 386616,
 395098,
 398162,
 401299,
 406390,
 410220,
 435253,
 448302,
 456814,
 477258,
 477294,
 477454,
 477651,
 477934,
 477939,
 477972,
 478037,
 478154,
 478442,
 478478,
 478627,
 478644,
 478655,
 478722,
 479145,
 479193,
 479210,
 543162,
 543177,
 543250,
 543450,
 543458,
 543688,
 544105,
 544404,
 544487,
 544700,
 545133,
 545224,
 545767,
 545771,
 546153,
 

In [35]:
human_in_dev = {k: v for k, v in true_dict.items() if k in human_ids}
human_in_dev

{}